In [1]:

include("../src/plasm.jl")
include("../src/read.jl")
include("../src/surface.jl")
include("../src/representation.jl")

using PyCall

In [2]:

@pyimport io3d

In [3]:
data = io3d.read("~/projects/io3d/sample_data/jatra_5mm/", dataplus_format=true)
data3d  = data["data3d"]
voxelsize_mm = data["voxelsize_mm"]

3-element Array{Any,1}:
 5.0 
 0.75
 0.75

In [4]:
data

Dict{Any,Any} with 9 entries:
  "voxelsize_mm"      => Any[5.0,0.75,0.75]
  "data3d"            => Int16[-1024 -1024 … -1024 -1024; -1024 -1024 … -1024 -…
  "datadir"           => "/home/mjirik/projects/io3d/sample_data/jatra_5mm"
  "Modality"          => "CT"
  "ImageComments"     => ""
  "SeriesNumber"      => 7
  "series_number"     => 7
  "dcmfilelist"       => Any["/home/mjirik/projects/io3d/sample_data/jatra_5mm/…
  "SeriesDescription" => "DE_Abdom_1F  5.0  B30f M_0.3"

In [5]:
data3d = data3d[:, 100:150, 100:140]

93×51×41 Array{Int16,3}:
[:, :, 1] =
   -70    -18    -37    -71    -82  …    -65    -68    -60    -50    -37
   -21    -49    -77    -86    -94        -6    -17    -16    -29    -54
   -71    -72    -82    -90    -97        52     52     44     38     53
   -81    -77    -74    -78    -83       -38    -46    -43    -27      1
   -70    -76    -74    -83    -97        43     70     66     62     69
   -93    -87    -96    -88    -90  …     13     11      8     -1    -17
  -107   -111   -105    -94    -93        -8    -63   -131   -218   -320
   -81   -100   -132    -99    -89      -356   -509   -630   -752   -829
   -95    -99    -96   -100   -114      -468   -614   -726   -812   -873
   -96   -112   -105    -97    -91      -255   -394   -509   -573   -654
  -108   -119   -113   -108   -105  …   -312   -360   -308   -209   -187
   -94   -102   -114   -117   -113       -82    -94    -54      3     42
  -102   -114   -108    -84    -73        47     38     41     38     57
     ⋮        

In [6]:
## Artifical data
data3d = zeros(Int16,3,3,3)
data3d[2:3,2,2] = 10
data3d

3×3×3 Array{Int16,3}:
[:, :, 1] =
 0  0  0
 0  0  0
 0  0  0

[:, :, 2] =
 0   0  0
 0  10  0
 0  10  0

[:, :, 3] =
 0  0  0
 0  0  0
 0  0  0

In [7]:
# useless

function thresholding(data3d, threshold)
    sz = size(data3d)
    segmentation = Array(Int8, size(data3d, 1), size(data3d, 2), size(data3d, 3))
    # produce vertexes
    for k in 1:sz[3]
        for j in 1:sz[2]
            for i in 1:sz[1]
                if data3d[i, j, k] > 0

                    segmentation[i, j, k] = data3d[i, j, k] > threshold
    #             if i == 0
    #             index = 1 + i + sz[1] * j + sz[1] * sz[2] * k
                end

            end
        end
    end
    return segmentation
end

thresholding (generic function with 1 method)

In [8]:
# segmentation
i = 1
j = 1
(i > 0) & (j > 0)

true

## Get vertices and count the voxels (* 4 = face number)

In [36]:
# Get all vertices and count the voxels
function get_index(sz, i, j, k) 
    index = 1 + i + (1 + sz[1]) * j + (sz[1] + 1) * (sz[2] + 1) * k 
    return index
end

function vertices_and_count_voxels_as_the_square_face_preprocessing(data3d, voxelsize_mm, threshold)
    """
    
    Fill the area with all possible vertices. Most of them will be removed in future.
    """
    threshold = 0
    ivertex = 0
    ifaces = 0
    sz = size(data3d)
    verts = Array(Float64, (sz[1]+1) * (sz[2]+1) * (sz[3]+1), 3)
    # produce vertexes
    nvoxels = 0
    for k in 0:sz[3]
        for j in 0:sz[2]
            for i in 0:sz[1]
                if (i > 0) & (j > 0) & (k > 0)
                    if data3d[i,j,k] > threshold
                        nvoxels += 1
                    end
                end
                pt = get_index(sz, i - 0, j -0, k -0)
    #             println(pt)
                verts[pt, 1] = i * voxelsize_mm[1]
                verts[pt, 2] = j * voxelsize_mm[2]
                verts[pt, 3] = k * voxelsize_mm[3]
            end
        end
    end
    return verts, nvoxels
end


#             if segmentation[i,j,k]

vertices_and_count_voxels_as_the_square_face_preprocessing (generic function with 2 methods)

In [37]:
verts, nvoxels = vertices_and_count_voxels_as_the_square_face_preprocessing(data3d, voxelsize_mm, threshold)
get_index(sz, 0, 0, 1)

17

In [11]:
verts

64×3 Array{Float64,2}:
  0.0  0.0   0.0 
  5.0  0.0   0.0 
 10.0  0.0   0.0 
 15.0  0.0   0.0 
  0.0  0.75  0.0 
  5.0  0.75  0.0 
 10.0  0.75  0.0 
 15.0  0.75  0.0 
  0.0  1.5   0.0 
  5.0  1.5   0.0 
 10.0  1.5   0.0 
 15.0  1.5   0.0 
  0.0  2.25  0.0 
  ⋮              
  0.0  0.75  2.25
  5.0  0.75  2.25
 10.0  0.75  2.25
 15.0  0.75  2.25
  0.0  1.5   2.25
  5.0  1.5   2.25
 10.0  1.5   2.25
 15.0  1.5   2.25
  0.0  2.25  2.25
  5.0  2.25  2.25
 10.0  2.25  2.25
 15.0  2.25  2.25

## Create face

In [40]:
#
#      
#       14 -------- 13
#     / |          / |
#   /   |        /   |          
# 11 --------- 12    |
# |     |       |    |        i 
# |     |       |    |               
# |     24 -----|-- 23        |     
# |   /         |   /         |   /
# | /           | /           | /
# 21 --------- 22             * ------ 
#
#


function create_square_faces(data3d, nvoxels, threshold)
    """
    create square faces based on vertices from vertices_and_count_voxels_as_...
    :nvoxels: number of voxels higher than the threslold.
    """
    nfaces = nvoxels * 6
    faces = Array(Int64, nfaces, 4)
    ifaces = 0
#     println(sz)
    # produce faces
    for k in 1:sz[3]
        for j in 1:sz[2]
            for i in 1:sz[1]
    #             println(i,",",j,",",k)
                if data3d[i,j,k] > threshold
    #                 println("jsme uvnitr")
                    pt11 = get_index(sz, i -0, j -0, k -0)
    #                 println("jsme uvnitr", pt11)
                    pt12 = get_index(sz, i -0, j -0, k -1)
                    pt13 = get_index(sz, i -0, j -1, k -1)
                    pt14 = get_index(sz, i -0, j -1, k -0)
                    pt21 = get_index(sz, i -1, j -0, k -0)
                    pt22 = get_index(sz, i -1, j -0, k -1)
                    pt23 = get_index(sz, i -1, j -1, k -1)
                    pt24 = get_index(sz, i -1, j -1, k -0)

                    faces[ifaces + 1, :] = [pt11, pt12, pt13, pt14]'
                    faces[ifaces + 2, :] = [pt22, pt23, pt13, pt12]
                    faces[ifaces + 3, :] = [pt21, pt22, pt12, pt11]
                    faces[ifaces + 4, :] = [pt14, pt13, pt23, pt24]
                    faces[ifaces + 5, :] = [pt11, pt14, pt24, pt21]
                    faces[ifaces + 6, :] = [pt21, pt24, pt23, pt22]
                    ifaces += 6

                end
            end
        end
    end
    return faces
end

create_square_faces (generic function with 1 method)

In [43]:
faces = create_square_faces(data3d, nvoxels, threshold)
# get_index(sz, 0,0,0)
faces

12×4 Array{Int64,2}:
 43  27  23  39
 26  22  23  27
 42  26  27  43
 39  23  22  38
 43  39  38  42
 42  38  22  26
 44  28  24  40
 27  23  24  28
 43  27  28  44
 40  24  23  39
 44  40  39  43
 43  39  23  27

## Double face removal

TODO

In [14]:
function arrayofarray2arrayd2d(d)
    data = Array(Float64,length(d),length(d[1]))
    for i in 1:length(d)
        for j in 1:length(d[1])
            data[i,j] = d[i][j]
        end
    end
    return data
end
function keep_surface_faces(vertexes, faces, index_base=0)
#     removeDoubleVertexesAndFaces
    println("step1")
    new_vertexes, inv_vertexes = remove_double_vertexes_alternative(vertexes)# , index_base=index_base)
    println("step2")
    new_faces = reindexVertexesInFaces(faces, inv_vertexes)
    println("step3")
    new_faces = removeDoubleFacesByAlberto(faces)
    println("step1")
    # todo remove unused vertexes
#     new_new_vertexes, new_new_inv_vertexes = remove_double_vertexes_alternative(new_vertexes)# , index_base=index_base)
    println("step2")
#     new_faces = reindexVertexesInFaces(faces, inv_vertexes)
    new_vertexes = arrayofarray2arrayd2d(new_vertexes)
    return new_vertexes, new_faces
end
new_verts, new_faces = keep_surface_faces(verts, faces)

step1
step2
step3
Removed face [1,12][43 42 38 39; 26 22 38 42; 27 26 42 43; 39 38 22 23; 43 39 23 27; 27 23 22 26; 59 58 54 55; 42 38 54 58; 43 42 58 59; 55 54 38 39; 59 55 39 43; 43 39 38 42]1
step1
step2


(
[0.0 0.0 0.0; 0.0 0.0 0.75; … ; 15.0 2.25 1.5; 15.0 2.25 2.25],

[43 42 58 59; 42 38 54 58; … ; 59 55 39 43; 43 39 23 27])

In [15]:
new_verts

64×3 Array{Float64,2}:
  0.0  0.0   0.0 
  0.0  0.0   0.75
  0.0  0.0   1.5 
  0.0  0.0   2.25
  0.0  0.75  0.0 
  0.0  0.75  0.75
  0.0  0.75  1.5 
  0.0  0.75  2.25
  0.0  1.5   0.0 
  0.0  1.5   0.75
  0.0  1.5   1.5 
  0.0  1.5   2.25
  0.0  2.25  0.0 
  ⋮              
 15.0  0.75  0.0 
 15.0  0.75  0.75
 15.0  0.75  1.5 
 15.0  0.75  2.25
 15.0  1.5   0.0 
 15.0  1.5   0.75
 15.0  1.5   1.5 
 15.0  1.5   2.25
 15.0  2.25  0.0 
 15.0  2.25  0.75
 15.0  2.25  1.5 
 15.0  2.25  2.25

In [20]:
verts = new_verts
faces = new_faces

10×4 Array{Int64,2}:
 43  42  58  59
 42  38  54  58
 39  38  22  23
 26  22  38  42
 27  26  42  43
 59  58  54  55
 27  23  22  26
 55  54  38  39
 59  55  39  43
 43  39  23  27

## Triangulation

In [25]:
function triangulation(faces)
    trifaces = Array(Int64, size(faces, 1) * 2, 3)
    for i in 1:size(faces,1)
        face = faces[i,:]
        trifaces[(i * 2) - 0, :] = [face[1], face[2], face[3]]
        trifaces[(i * 2) - 1, :] = [face[1], face[3], face[4]]
    end
    return trifaces
end

triangulation (generic function with 1 method)

In [26]:
trifaces = triangulation(faces)

20×3 Array{Int64,2}:
 43  58  59
 43  42  58
 42  54  58
 42  38  54
 39  22  23
 39  38  22
 26  38  42
 26  22  38
 27  42  43
 27  26  42
 59  54  55
 59  58  54
 27  22  26
 27  23  22
 55  38  39
 55  54  38
 59  39  43
 59  55  39
 43  23  27
 43  39  23

## Visualization

In [24]:
check_vf(verts, trifaces)
visualize_numbers(to_lar(verts, trifaces), 2)

vertex index index higher than voxel number
faces min 22, faces max 59, vertex number 64
43
58
58
54
22
22
38
22
42
26
54
59
22
23
38
54
43
59
27
39
XgeModule::init_started
Creating shared GLCanvas...
shared GLCanvas created
XgeModule::init_finished
Evaluating fenvs.py..
('...fenvs.py imported in', 0.007949999999999902, 'seconds')
centre
centre
centre
Building batches from Hpc....
...done in 13 msec
Optimizing the octree....
   Number of input batches 1488
   total number vertices    262
   Number of output batches 131
   Batch vertex media       2
...done in 1 msec
Building octree from 131 batches....
Scene number of nodes of the octree 139
Scene max depth                     4
Scene number of batches             131
...done in 0 msec


In [19]:
uuu = [1,2]
size(uuu,1)

2